In [1]:
import csv
import glob
import os
import time
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from sqlalchemy.dialects import registry
import boto3
import pandas as pd
import snowflake.connector as sf
from snowflake.connector.pandas_tools import write_pandas

ModuleNotFoundError: No module named 'snowflake.sqlalchemy'

In [2]:
# log
start_time = time.time()
auth_log = []

In [3]:
try:
    # Create local directory to store downloaded files
    os.makedirs('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CCAFILES\\CCA_AUTH')
    os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CCAFILES\\CCA_AUTH')
    root_directory = os.getcwd()
    auth_log.append('Successfully created CCA_AUTH File Temporary Folder at ' + time.strftime('%Y-%m-%d %H:%M:%S',
                                                                                              time.localtime(
                                                                                                  time.time())))
except Exception as e:
    auth_log.append('There was an error with creating the temporary CCA_AUTH File - ' + str(e))
    print('Successfully created CCA_AUTH File Temporary Folder at ' + time.strftime('%Y-%m-%d %H:%M:%S',
                                                                                    time.localtime(time.time())))


Successfully created CCA_AUTH File Temporary Folder at 2022-12-13 15:17:44


In [4]:
#Create Connection Object for Connecting to AWS

try:
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1',
        aws_access_key_id=os.getenv('aws_access_key_id'),
        aws_secret_access_key=os.getenv('aws_secret_access_key'))
    print('AWS connection object created at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    auth_log.append(
        'AWS connection object created at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
except Exception as e:
    print('There was an error with creating the AWS connection object - ' + str(e))
    auth_log.append('There was an error with creating the AWS connection object - ' + str(e))


AWS connection object created at 2022-12-13 15:17:46


In [9]:
#Create the file name format for locating the proper CCA files to parse
MONTH = '12'

filename_format_list = ['Auth Universe Report_' + MONTH]
auth_log.append('Looking for CCA Files that start like ' + " and ".join(filename_format_list) + ' at ' + time.strftime(
    '%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
print('Looking for CCA Files that start like ' + " and ".join(filename_format_list) + ' at ' + time.strftime(
    '%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

Looking for CCA Files that start like Auth Universe Report_12 at 2022-12-13 15:20:13


In [10]:
#Creating a list to store all the keys (file names) to download

key_list = []

try:
    #Searching the S3 bucket for the most current Ping Files
    for obj in s3.Bucket('hometeam-clinical-data').objects.all():
        for filename_format in filename_format_list:
            if filename_format in str(obj):
                #print(obj.key)
                key_list.append(obj.key)

except Exception as e:
    auth_log.append('There was an error while looking for most CCA Files - ' + str(e))

In [11]:
print(key_list)

['raw/cca/Auth Universe Report_12.12.2022.xlsx']


In [12]:
try:
    #Downloading each of the files found in the key list
    for file in key_list:
        s3.Bucket('hometeam-clinical-data').download_file(file, file.split('/')[2])
        print(
            'Files were successfully downloaded at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    auth_log.append(
        'Files were successfully downloaded at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
except Exception as e:
    auth_log.append('There was an error while trying to download the CCA Files - ' + str(e))

Files were successfully downloaded at 2022-12-13 15:20:23


In [13]:


#Set current directory
cwd = os.chdir('C:\\Users\\Jad Driggers\\Documents\\CCA_AUTH')

for i, file in enumerate(os.listdir(cwd)):
    if '.xlsx' in file:
        df_list = []

        with open(file, 'r') as csvfile:
            reader = csv.reader(csvfile)

    print(csvfile)

<_io.TextIOWrapper name='Auth Universe Report_12.05.2022.xlsx' mode='r' encoding='cp1252'>


In [14]:
# % % time
# #Create dictionary to store dataframes as they are created
# df_dict = {}
# error_dict = {}
# use glob to get all the csv files
# in the folder
path = os.getcwd()
#print(os.getcwd())
csv_files = glob.glob(os.path.join(path, "*.xlsx"))
#cwd = os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CCAFILES\\CCA_AUTH')

# Loop over files
for f in csv_files:
    df = pd.read_excel(f)

    # print the location and filename
    pd.set_option('display.max_colwidth', None)
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # print the content
    print('Content:')
    display(df)
    print()







Location: C:\Users\Jad Driggers\Documents\CCA_AUTH\Auth Universe Report_12.05.2022.xlsx
File Name: Auth Universe Report_12.05.2022.xlsx
Content:


,AUTH_ID,CCAID,Product,cat,auth_priority,Received_Date,min_SERVICE_FROM_DATE,max_SERVICE_to_DATE,Referred To/Facility,Service Code(s),...,Decision_Date,WrittenNotifDate,deadline,TAT,CMO,Verbal_Notification_Fixed_New,Letter Flag,Admin Approval or Admin Denial,LoadDate,Altruista_updatedate
0,0102S1Q6D,5366000712,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-01 19:00:00.979,2022-01-07,2022-03-07,"Comfort Home Care, LLC",G0299,...,2022-01-03 09:55:42.038,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
1,0102S38PQ,5365706182,Senior Care Options (SCO)-MA,Procedure,Prospective Expedited,2022-01-02 12:04:03.994,2022-01-04,2023-01-04,Matthew Jonathan Weinstock,J9312,...,2022-01-02 21:45:54.000,2022-01-03 12:00:00.000,2022-01-06,24 hours,Element Care,2022-01-03 07:59:59.971,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
2,0102SMODE,5365665706,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-01 18:22:10.042,2022-01-06,2022-03-06,"Comfort Home Care, LLC",G0299,...,2022-01-03 09:03:16.042,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
3,0102SZTF9,5365576687,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-02 09:42:14.976,2021-12-28,2022-02-25,Ideal Health Care System,G0299,...,2022-01-03 10:37:53.040,2022-01-03 16:59:59.971,2022-01-16,1 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
4,0103M2N3I,5364525974,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-03 12:19:37.027,2021-12-25,2022-02-22,"Suburban Home Health Care, Inc.",G0299,...,2022-01-04 08:50:18.010,2022-01-04 16:59:59.971,2022-01-17,1 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11096,1231FCFJB,5365652676,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:22:21.965,2022-01-01,2022-01-27,"Quality Care, Inc.",S5135,...,2022-01-03 09:46:11.971,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
11097,1231FRDSP,5365612416,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:33:33.034,2022-01-01,2022-06-30,Uniting With You Home Care LLC,S5130,...,2022-01-03 08:59:49.027,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
11098,1231FSXXZ,5365584766,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2021-12-31 12:53:09.024,2022-01-01,2022-01-28,"All Care VNA of Greater Lynn, Inc.",G0299,...,2022-01-03 09:55:37.978,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
11099,1231FV5QD,5365652676,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:30:20.016,2022-01-01,2022-01-27,"Quality Care, Inc.",S5130 | S5131,...,2022-01-03 09:49:19.027,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030


In [131]:
bucket = 'hometeam-clinical-data'
copy_source = {
    'Bucket': bucket,
    'Key': 'raw/cca/Element MDS 202209.zip',
}

copy_source2 = {
    'Bucket': bucket,
    'Key': 'raw/cca/Element claims 202209.zip',
}
s3.meta.client.copy(copy_source, bucket, 'raw/cca/Monthly_Claims/Element MDS 202209.zip')
s3.meta.client.copy(copy_source2, bucket, 'raw/cca/Monthly_Claims/Element claims 202209.zip')
s3_object = s3.Object('hometeam-clinical-data', 'raw/cca/Monthly_Claims/Element MDS 202209.zip')
s3_object.delete()

NameError: name 's3' is not defined

In [15]:
# Regex replace Service Description(s) extra char

df = df.replace(r'\W+', ' ', regex=True)
# df['SERVICEDESCRIPTION'].str.slice(0,20)

#df['SERVICEDESCRIPTION'].str.len()

In [81]:

df.head()

,AUTHID,MEMBERID,PRODUCT,CATEGORY,AUTHPRIORITY,RECEIVEDDATE,SERVICEFROMDATE,SERVICETODATE,REFERREDTO,SERVICECODE,...,DECISIONDATE,WRITTENNOTIFDATE,DEADLINE,TAT,CMO,VERBALNOTIF,LETTERFLAG,APPROVEDENIED,LOADDATE,ALTRUISTADATE
0,0102S1Q6D,5366000712,Senior Care Options SCO MA,Home Health,Prospective Standard,2022-01-01 19:00:00.979,2022-01-07,2022-03-07,Comfort Home Care LLC,G0299,...,2022-01-03 09:55:42.038,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
1,0102S38PQ,5365706182,Senior Care Options SCO MA,Procedure,Prospective Expedited,2022-01-02 12:04:03.994,2022-01-04,2023-01-04,Matthew Jonathan Weinstock,J9312,...,2022-01-02 21:45:54.000,2022-01-03 12:00:00.000,2022-01-06,24 hours,Element Care,2022-01-03 07:59:59.971,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
2,0102SMODE,5365665706,Senior Care Options SCO MA,Home Health,Prospective Standard,2022-01-01 18:22:10.042,2022-01-06,2022-03-06,Comfort Home Care LLC,G0299,...,2022-01-03 09:03:16.042,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
3,0102SZTF9,5365576687,Senior Care Options SCO MA,Home Health,Prospective Standard,2022-01-02 09:42:14.976,2021-12-28,2022-02-25,Ideal Health Care System,G0299,...,2022-01-03 10:37:53.040,2022-01-03 16:59:59.971,2022-01-16,1 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030
4,0103M2N3I,5364525974,Senior Care Options SCO MA,Home Health,Prospective Standard,2022-01-03 12:19:37.027,2021-12-25,2022-02-22,Suburban Home Health Care Inc,G0299,...,2022-01-04 08:50:18.010,2022-01-04 16:59:59.971,2022-01-17,1 day,Element Care,NaT,TimelyLetter,No,2022-12-05 06:37:21.994,2022-12-04 23:04:41.030


In [ ]:
df = df.rename(columns={"AUTH_ID": "AUTHID",
                        "CCAID": "MEMBERID",
                        "Product": "PRODUCT",
                        "cat": "CATEGORY",
                        "auth_priority": "AUTHPRIORITY",
                        "Received_Date": "RECEIVEDDATE",
                        "min_SERVICE_FROM_DATE": "SERVICEFROMDATE",
                        "max_SERVICE_to_DATE": "SERVICETODATE",
                        "Referred To/Facility": "REFERREDTO",
                        "Service Code(s)": "SERVICECODE",
                        "Service Description(s)": "SERVICEDESCRIPTION",
                        "IS_extension": "ISEXTENSION",
                        "OverallDecision": "OVERALLDECISION",
                        "Decision_Date": "DECISIONDATE",
                        "WrittenNotifDate": "WRITTENNOTIFDATE",
                        "deadline": "DEADLINE",
                        "TAT": "TAT",
                        "CMO": "CMO",
                        "Verbal_Notification_Fixed_New": "VERBALNOTIF",
                        "Letter Flag": "LETTERFLAG",
                        "Admin Approval or Admin Denial": "APPROVEDENIED",
                        "LoadDate": "LOADDATE",
                        "Altruista_updatedate": "ALTRUISTADATE", })

In [105]:
#Create connection to Snowflake using your account and user


account_identifier = os.getenv('Snowflake_account')
user = os.getenv('Snowflake_User')
password = os.getenv('Snowflake_password')

conn_string = f"snowflake://{user}:{password}@{account_identifier}/"
engine = create_engine(conn_string)

ModuleNotFoundError: No module named 'snowflake.sqlalchemy'

In [73]:
start_time = time.time()
#Creating of parameters for securing connection to Snowflake-credentials stored in local environment variables
username = os.getenv('Snowflake_User')
password = os.getenv('Snowflake_password')
account = os.getenv('Snowflake_account')

#Define parameters if neccessary
warehouse = 'DEVELOPER_STANDARD'
database = 'VESTA_DEVELOPMENT'
schema = 'ANALYST_SANDBOX'

#Create connection object for Snowflake connection
conn = sf.connect(user=username, password=password, account=account, warehouse=warehouse)


#Execution function
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close


#Define Database to use in Snowflake
sql = 'USE DATABASE {}'.format(database)
execute_query(conn, sql)

#Define Schema to use in Snowflake
sql = 'USE SCHEMA {}.{}'.format(database, schema)
execute_query(conn, sql)

#Define Warehouse to use in Snowflake
sql = 'USE WAREHOUSE {}'.format(warehouse)
execute_query(conn, sql)

print('Successful Connection')

Successful Connection


In [77]:

try:
    sql = 'DROP TABLE IF EXISTS CCA_AUTH_RAW_TEST_NEW'
    execute_query(conn, sql)

except Exception as e:
    print(e)


In [ ]:
try:
    sql = 'CREATE TABLE IF NOT EXISTS CCA_AUTH_RAW_TEST_NEW'
    execute_query(conn, sql)

except Exception as e:
    print(e)


In [ ]:
try:
    sql = '''COPY INTO CCA_AUTH_RAW_TEST_NEW'''
    execute_query(conn, sql)

    success, nchucks, nrows, _ = df.to_sql('CCA_AUTH_RAW_TEST_NEW',conn, df)

except Exception as e:
    print(e)
print('DataFrame successfully appended to Auth Table in Snowflake at ' + time.strftime('%Y-%m-%d %H:%M:%S',
                                                                                       time.localtime(time.time())))

In [78]:
try:
    sql = '''CREATE TABLE IF NOT EXISTS CCA_AUTH_RAW_TEST_NEW (AUTHID VARCHAR(200), MEMBERID VARCHAR(200), PRODUCT VARCHAR(200), CATEGORY VARCHAR(200), AUTHPRIORITY VARCHAR(200), RECEIVEDDATE VARCHAR(200), SERVICEFROMDATE VARCHAR(200), SERVICETODATE VARCHAR(200), REFERREDTO VARCHAR(200), SERVICECODE VARCHAR(200), SERVICEDESCRIPTION VARCHAR(2200), ISEXTENSION VARCHAR(200), OVERALLDECISION VARCHAR(200), DECISIONDATE VARCHAR(200),  WRITTENNOTIFDATE VARCHAR(200), DEADLINE VARCHAR(200), TAT VARCHAR(200), CMO VARCHAR(200), VERBALNOTIF VARCHAR(200), LETTERFLAG VARCHAR(200), APPROVEDENIED VARCHAR(200), LOADDATE VARCHAR(200), ALTRUISTADATE VARCHAR(200))'''
    execute_query(conn, sql)

    success, nchucks, nrows, _ = write_pandas(conn, df, 'CCA_AUTH_RAW_TEST_NEW')

except Exception as e:
    print(e)
print('DataFrame successfully appended to Auth Table in Snowflake at ' + time.strftime('%Y-%m-%d %H:%M:%S',
                                                                                       time.localtime(time.time())))

DataFrame successfully appended to Auth Table in Snowflake at 2022-12-13 16:34:40


In [79]:
# UPDATE VESTA_DEVELOPMENT.ANALYST_SANDBOX.CCA_AUTH_RAW_TEST_NEW
# set receiveddate to_timestamp(receiveddate),
# servicefromdate to_timestamp(servicefromdate),
# servicetodate to_timestamp(servicetodate),
# decisiondate to_timestamp(decisiondate),
# writtennotifdate = to_timestamp(writtennotifdate),
#     deadline = to_timestamp(deadline),
#     loaddate = to_timestamp(loaddate),
#     altruistadate = to_timestamp(altruistadate)


ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 2 at position 17 unexpected 'to_timestamp'.
syntax error line 3 at position 16 unexpected 'to_timestamp'.
syntax error line 4 at position 14 unexpected 'to_timestamp'.
syntax error line 5 at position 13 unexpected 'to_timestamp'.

In [27]:
    # Query to Snowflake
try:
    sql = 'CREATE TABLE IF NOT EXISTS CCA_AUTH_RAW_TEST_NEW'
    # sql = '''CREATE TABLE IF NOT EXISTS CCA_AUTH_RAW_TEST_NEW (AUTHID VARCHAR(200), MEMBERID VARCHAR(200), PRODUCT VARCHAR(200), CATEGORY VARCHAR(200), AUTHPRIORITY VARCHAR(200), RECEIVEDDATE DATETIME, SERVICEFROMDATE TIMESTAMP, SERVICETODATE VARCHAR(200), REFERREDTO VARCHAR(200), SERVICECODE VARCHAR(200), SERVICEDESCRIPTION VARCHAR(2200), ISEXTENSION VARCHAR(200), OVERALLDECISION VARCHAR(200), DECISIONDATE VARCHAR(200),  WRITTENNOTIFDATE VARCHAR(200), DEADLINE VARCHAR(200), TAT VARCHAR(200), CMO VARCHAR(200), VERBALNOTIF VARCHAR(200), LETTERFLAG VARCHAR(200), APPROVEDENIED VARCHAR(200), LOADDATE VARCHAR(200), ALTRUISTADATE VARCHAR(200))'''
    execute_query(conn, sql)
    success, nchucks, nrows, _ = write_pandas(conn, df, 'CCA_AUTH_RAW_TEST_NEW')
except Exception as e:
    print(e)
print('DataFrame successfully appended to Auth Table in Snowflake at ' + time.strftime('%Y-%m-%d %H:%M:%S',
                                                                                       time.localtime(time.time())))

IndentationError: expected an indented block (4196269275.py, line 3)

In [22]:

conn.close



# except Exception as e:
#         print(e)

finally:
conn.close

Successful Connection
001003 (42000): SQL compilation error:
syntax error line 1 at position 48 unexpected '<EOF>'.
DataFrame successfully appended to Auth Table in Snowflake at 2022-12-13 15:26:03


In [135]:
df.to_csv('Auth_test15.csv')

0                                Skilled Nursing 60 Days
1                              Injection rituximab 10 mg
2                                Skilled Nursing 60 Days
3                                Skilled Nursing 60 Days
4                                Skilled Nursing 60 Days
                              ...                       
11641                                 Companion Services
11642                                  Homemaker Service
11643                            Skilled Nursing 60 Days
11644    Homemaker Service Personal Care Agency not PCA)
11645           Physical Therapy Skilled Nursing 60 Days
Name: Service Description(s), Length: 11646, dtype: object

In [35]:
df

,AUTH_ID,MEMBER_ID,Product,cat,auth_priority,Received_Date,SERVICE_FROM_DATE,SERVICE_TO_DATE,Referred To/Facility,Service Code(s),...,Decision_Date,WrittenNotifDate,deadline,TAT,CMO,Verbal_Notification_Fixed_New,Letter Flag,Admin Approval or Admin Denial,LoadDate,Altruista_updatedate
0,0102S1Q6D,5366000712,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-01 19:00:00.979,2022-01-07,2022-03-07,"Comfort Home Care, LLC",G0299,...,2022-01-03 09:55:42.038,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
1,0102S38PQ,5365706182,Senior Care Options (SCO)-MA,Procedure,Prospective Expedited,2022-01-02 12:04:03.994,2022-01-04,2023-01-04,Matthew Jonathan Weinstock,J9312,...,2022-01-02 21:45:54.000,2022-01-03 12:00:00.000,2022-01-06,24 hours,Element Care,2022-01-03 07:59:59.971,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
2,0102SMODE,5365665706,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-01 18:22:10.042,2022-01-06,2022-03-06,"Comfort Home Care, LLC",G0299,...,2022-01-03 09:03:16.042,2022-01-03 16:59:59.971,2022-01-15,2 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
3,0102SZTF9,5365576687,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-02 09:42:14.976,2021-12-28,2022-02-25,Ideal Health Care System,G0299,...,2022-01-03 10:37:53.040,2022-01-03 16:59:59.971,2022-01-16,1 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
4,0103M2N3I,5364525974,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2022-01-03 12:19:37.027,2021-12-25,2022-02-22,"Suburban Home Health Care, Inc.",G0299,...,2022-01-04 08:50:18.010,2022-01-04 16:59:59.971,2022-01-17,1 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,1231FCFJB,5365652676,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:22:21.965,2022-01-01,2022-01-27,"Quality Care, Inc.",S5135,...,2022-01-03 09:46:11.971,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
11642,1231FRDSP,5365612416,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:33:33.034,2022-01-01,2022-06-30,Uniting With You Home Care LLC,S5130,...,2022-01-03 08:59:49.027,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
11643,1231FSXXZ,5365584766,Senior Care Options (SCO)-MA,Home Health,Prospective Standard,2021-12-31 12:53:09.024,2022-01-01,2022-01-28,"All Care VNA of Greater Lynn, Inc.",G0299,...,2022-01-03 09:55:37.978,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
11644,1231FV5QD,5365652676,Senior Care Options (SCO)-MA,LTSS,Prospective Standard,2021-12-31 06:30:20.016,2022-01-01,2022-01-27,"Quality Care, Inc.",S5130 | S5131,...,2022-01-03 09:49:19.027,2022-01-03 16:59:59.971,2022-01-14,3 day,Element Care,NaT,TimelyLetter,No,2022-10-10 06:24:54.029,2022-10-09 22:18:33.034
